In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DGX', 'DIS', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY',
    'FLGT', 'FTNT', 'GH', 'GMED', 'GOOG', 'IDXX', 'INTG', 'ISRG',
    'JD', 'KNSL', 'LRCX', 'LSPD', 'LULU', 'MA', 'MASI', 'MDB',
    'MELI', 'MNST', 'MSFT', 'MTCH', 'NET', 'NOW', 'NVCR', 'NVDA',
    'ODFL', 'OKTA', 'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON',
    'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD', 'ROKU',
    'RVLV', 'SAM', 'SBUX', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC',
    'TEAM', 'TMUS', 'TREX', 'TSCO', 'TSLA', 'TTC', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV',
    'ZBRA', 'ZM']
LINGERERS = [
    'AGI', 'AMEH', 'APPS', 'APT', 'BRTX', 'BTTR', 'BYRN', 'CEF', 'CENN',
    'CGAU', 'COOP', 'CTGO', 'DQ', 'DTST', 'DYAI', 'EA',
    'EP', 'EPSN',  'ESOA', 'FCUV', 'FNV', 'FSI', 'GCBC', 'GME',
    'GNRC', 'GTLS', 'HBM', 'HMY', 'HNRG', 'IDR', 'INTZ', 'LPLA', 'MCVT',
    'MICS', 'MNMD', 'MOBQ', 'MTA', 'OMQS', 'PACB', 'PETV', 'POWW',
    'PRPH', 'RVP', 'RVYL', 'SAND', 'SOBR', 'STKL', 'UI', 'VIRT', 'WAVD',
    'WKHS', 'WRN', 'ZEST', 'ZYXI']
BEST_WEIGHTED = [
    'AORT', 'ATVI', 'BHC', 'CHS', 'CNXN', 'CRMT', 'CRUS', 'CUBI',
    'DORM', 'FKWL', 'LFMD', 'PARR', 'POWL', 'SIX', 'SRC', 'URBN',
    'USPH', 'VERU', 'VTSI', 'WGO']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))
len(SYMBOLS)

164

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ACN
ADBE


### E*Trade

In [10]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [11]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
003CVR016,--
AAPL,3553.5
ACN,2313.76
ADBE,3793.2999999999997
ADYEY,2977.33


In [12]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
003CVR016,--
AAPL,3554
ACN,2314
ADBE,3793
ADYEY,2977


In [13]:
#et_df.drop(index=['98986CONT'], inplace=True)
et_df.head()

,ET
003CVR016,--
AAPL,3554
ACN,2314
ADBE,3793
ADYEY,2977


In [14]:
#et_df.drop('98936J101', inplace=True)

In [15]:
et_df.loc[et_df.ET == '--']
#et_df.loc['ABMD', 'ET'] = 380.11
et_df.drop('003CVR016', inplace=True)

In [16]:
check_symbols(et_df)

All clear


In [17]:
#et_df = et_df[1:]

In [18]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,3554
ABMD,0
ABNB,0
ACN,2314
ADBE,3793


### Fidelity

In [19]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Feb-03-2023.csv'

In [20]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv
Portfolio Performance - Damian Satterthwaite-Phillips.csv
Portfolio_Positions_Feb-03-2023.csv
Positions.csv
Positions_damiansp_Stocks_2023_02_03.xls


In [21]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-30-", "-31-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,5022.76,$1.00,$0.00,$5022.76,NaN,NaN,NaN,NaN,12.99%,NaN,NaN,Cash
1,218520300,Rollover IRA,003CVR016,ABIOMED INC SHS,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00%,--,--,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$17.21,-$0.568,$223.73,-$7.39,-3.20%,+$9.04,+4.21%,0.58%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.00,$86.09,-$2.22,$1549.62,-$39.96,-2.52%,+$202.71,+15.05%,4.01%,$1346.91,$74.83,Cash
4,218520300,Rollover IRA,AMGN,AMGEN INC,28.00,$245.17,-$1.36,$6864.76,-$38.08,-0.56%,-$69.94,-1.01%,17.76%,$6934.70,$247.67,Cash


In [22]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
2,Rollover IRA,ADYEY,224
3,Rollover IRA,AMD,1550
4,Rollover IRA,AMGN,6865
5,Rollover IRA,BROS,39
6,Rollover IRA,BYND,78


In [23]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ADYEY,224
AMD,1550
AMGN,6865
BROS,39
BYND,78


In [24]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [25]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,3554,0,618,0
ABMD,0,0,0,0
ABNB,0,0,119,0
ACN,2314,0,1735,0
ADBE,3793,0,0,0


### TD Ameritrade

In [26]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [27]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

#path = f'{DL}/Positions_damiansp_Stocks_2022_11_04.xls'

In [28]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1].replace(',', ''))
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [29]:
td = parse_file(path)
td.head()

,TD
AMEH,145.40
AMGN,980.68
AMZN,103.14
APPS,86.50
BRTX,6.28


In [30]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,3554,0,618,0,0
ABMD,0,0,0,0,0
ABNB,0,0,119,0,0
ACN,2314,0,1735,0,0
ADBE,3793,0,0,0,0


In [31]:
rsi = pd.read_csv(
    '~/Downloads/Holdings - Damian Satterthwaite-Phillips.csv',
    index_col=0,
    usecols=['Symbol', 'Value'])
rsi.Value = rsi.Value.apply(
    lambda x: round(float(x.replace('$', '').replace(',', ''))))
out = pd.concat([out, rsi], axis=1)
out.rename(columns={'Value': 'RSI'}, inplace=True)
out['RSI'] = out.RSI.fillna(0).astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD,RSI
AAPL,3554.0,0.0,618.0,0.0,0.0,1390
ABMD,0.0,0.0,0.0,0.0,0.0,0
ABNB,0.0,0.0,119.0,0.0,0.0,0
ACN,2314.0,0.0,1735.0,0.0,0.0,2025
ADBE,3793.0,0.0,0.0,0.0,0.0,759


In [32]:
out.to_csv(f'{DL}/clean.csv')

In [33]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*